# Model Exploration

In [2]:
import numpy as np
import pandas as pd
import spacy
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
nlp = spacy.load('en_core_web_lg')

In [5]:
training_df = pd.read_csv('data/clean_text.csv')
training_df.head()

,intensity,polarity,text
0,1.0,0.0,Don't buy: This company will set you back a bi...
1,0.0,0.5,"experimentation or other means), also called e..."
2,1.0,1.0,Beaugh: Excellent basic material. Well organiz...
3,0.0,0.5,"""An eye for an eye"", ""a tooth for a tooth"" or ..."
4,1.0,1.0,Can it get any better?: I have not read all th...


In [11]:
training_df['polarity'].value_counts(normalize=True)

0.5    0.5006
0.0    0.2538
1.0    0.2456
Name: polarity, dtype: float64

In [12]:
training_df['int_high'] = training_df['intensity'].apply(lambda x: 1 if x > 0.6 else 0)
training_df['int_med'] = training_df['intensity'].apply(lambda x: 1 if (x < 0.6 and x > 0.4) else 0)
training_df['int_low'] = training_df['intensity'].apply(lambda x: 1 if x < 0.4 else 0)
training_df['pol_pos'] = training_df['polarity'].apply(lambda x: 1 if x > 0.6 else 0)
training_df['pol_neu'] = training_df['polarity'].apply(lambda x: 1 if (x < 0.6 and x > 0.4) else 0)
training_df['pol_neg'] = training_df['polarity'].apply(lambda x: 1 if x < 0.4 else 0)

In [14]:
print(training_df['int_high'].value_counts(normalize=True))
print(training_df['int_med'].value_counts(normalize=True))
print(training_df['int_low'].value_counts(normalize=True))
print(training_df['pol_pos'].value_counts(normalize=True))
print(training_df['pol_neu'].value_counts(normalize=True))
print(training_df['pol_neg'].value_counts(normalize=True))

0    0.6664
1    0.3336
Name: int_high, dtype: float64
0    0.9382
1    0.0618
Name: int_med, dtype: float64
1    0.6046
0    0.3954
Name: int_low, dtype: float64
0    0.7544
1    0.2456
Name: pol_pos, dtype: float64
1    0.5006
0    0.4994
Name: pol_neu, dtype: float64
0    0.7462
1    0.2538
Name: pol_neg, dtype: float64


In [15]:
training_df['vector'].values[0].shape

(300,)

## Creating a basic model

In [37]:
model = keras.Sequential()
model.add(keras.layers.Dense(300, activation=tf.nn.relu))
model.add(keras.layers.Dense(32, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.sigmoid))

In [38]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
input_data = np.stack(training_df['vector'].values, axis=0)
input_data.shape

(10000, 300)

In [40]:
label_data = training_df[['int_high', 'int_med', 'int_low', 'pol_pos', 'pol_neu', 'pol_neg']].values
label_data.shape

(10000, 6)

In [41]:
split_frac = 0.2
cutoff = int(len(input_data) * split_frac)
x_val = input_data[:cutoff]
x_train = input_data[cutoff:]

y_val = label_data[:cutoff]
y_train = label_data[cutoff:]

In [42]:
history = model.fit(x_train,
                    y_train,
                    epochs=80,
                    batch_size=1024,
                    validation_data=(x_val, y_val))

Train on 8000 samples, validate on 2000 samples
Epoch 1/80
8000/8000 [==============================] - 1s 65us/step - loss: 0.6509 - acc: 0.6524 - val_loss: 0.5870 - val_acc: 0.7554
Epoch 2/80
8000/8000 [==============================] - 0s 7us/step - loss: 0.5544 - acc: 0.7494 - val_loss: 0.5187 - val_acc: 0.7502
Epoch 3/80
8000/8000 [==============================] - 0s 7us/step - loss: 0.5022 - acc: 0.7570 - val_loss: 0.4773 - val_acc: 0.7808
Epoch 4/80
8000/8000 [==============================] - 0s 7us/step - loss: 0.4640 - acc: 0.7808 - val_loss: 0.4494 - val_acc: 0.7841
Epoch 5/80
8000/8000 [==============================] - 0s 9us/step - loss: 0.4396 - acc: 0.7872 - val_loss: 0.4301 - val_acc: 0.7932
Epoch 6/80
8000/8000 [==============================] - 0s 8us/step - loss: 0.4185 - acc: 0.7977 - val_loss: 0.4138 - val_acc: 0.8026
Epoch 7/80
8000/8000 [==============================] - 0s 8us/step - loss: 0.4012 - acc: 0.8095 - val_loss: 0.4008 - val_acc: 0.8114
Epoch 8/80
80

In [28]:
def readable_output(output):
    intensity = output[0][:3]
    polarity = output[0][3:]
    pol_str = ""
    int_str = ""
    if max(intensity) == intensity[0]:
        int_str = "high"
    elif max(intensity) == intensity[1]:
        int_str = "med"
    else:
        int_str = "low"
    if max(polarity) == polarity[0]:
        pol_str = "pos"
    elif max(polarity) == polarity[1]:
        pol_str = "neutral"
    else:
        pol_str = "neg"
    return "{} {}".format(int_str, pol_str)

In [33]:
docs = [
    "I really hate this",
    "I don't like this",
    "I think this is nice",
    "I love this thing so much",
    "I feel relatively indifferent",]
for doc in docs:
    result = model.predict(np.asmatrix(nlp(doc).vector))
    print(result)
    print(doc, "\n", readable_output(result), "\n")

[[0.9006553  0.01879829 0.02905652 0.32951632 0.00907019 0.57859606]]
I really hate this 
 high neg 

[[0.9238026  0.05397011 0.02437352 0.41188955 0.00351907 0.7160095 ]]
I don't like this 
 high neg 

[[0.89268696 0.01106873 0.04471983 0.81357646 0.00932928 0.1418184 ]]
I think this is nice 
 high pos 

[[0.5061053  0.05087366 0.23811758 0.8225473  0.02424105 0.0759644 ]]
I love this thing so much 
 high pos 

[[0.0276257  0.00144556 0.97583145 0.00902619 0.79991555 0.1624663 ]]
I feel relatively indifferent 
 low neutral 



## Simpler Labels

In [49]:
model = keras.Sequential()
model.add(keras.layers.Dense(300, activation=tf.nn.relu))
model.add(keras.layers.Dense(32, activation=tf.nn.relu))
model.add(keras.layers.Dense(2, activation=tf.nn.sigmoid))

In [53]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['mae'])

In [54]:
input_data = np.stack(training_df['vector'].values, axis=0)
input_data.shape

(10000, 300)

In [55]:
label_data = training_df[['intensity', 'polarity']].values
label_data.shape

(10000, 2)

In [56]:
split_frac = 0.2
cutoff = int(len(input_data) * split_frac)
x_val = input_data[:cutoff]
x_train = input_data[cutoff:]

y_val = label_data[:cutoff]
y_train = label_data[cutoff:]

In [57]:
history = model.fit(x_train,
                    y_train,
                    epochs=80,
                    batch_size=1024,
                    validation_data=(x_val, y_val))

Train on 8000 samples, validate on 2000 samples
Epoch 1/80
8000/8000 [==============================] - 1s 70us/step - loss: 0.3743 - mean_absolute_error: 0.1450 - val_loss: 0.6916 - val_mean_absolute_error: 0.2477
Epoch 2/80
8000/8000 [==============================] - 0s 8us/step - loss: 0.3377 - mean_absolute_error: 0.1289 - val_loss: 0.6568 - val_mean_absolute_error: 0.2448
Epoch 3/80
8000/8000 [==============================] - 0s 7us/step - loss: 0.3189 - mean_absolute_error: 0.1149 - val_loss: 0.6429 - val_mean_absolute_error: 0.2372
Epoch 4/80
8000/8000 [==============================] - 0s 7us/step - loss: 0.3101 - mean_absolute_error: 0.1086 - val_loss: 0.6543 - val_mean_absolute_error: 0.2392
Epoch 5/80
8000/8000 [==============================] - 0s 9us/step - loss: 0.3077 - mean_absolute_error: 0.1068 - val_loss: 0.6522 - val_mean_absolute_error: 0.2386
Epoch 6/80
8000/8000 [==============================] - 0s 7us/step - loss: 0.3050 - mean_absolute_error: 0.1045 - val_lo

Epoch 50/80
8000/8000 [==============================] - 0s 8us/step - loss: 0.2622 - mean_absolute_error: 0.0722 - val_loss: 0.7449 - val_mean_absolute_error: 0.2391
Epoch 51/80
8000/8000 [==============================] - 0s 8us/step - loss: 0.2606 - mean_absolute_error: 0.0718 - val_loss: 0.7503 - val_mean_absolute_error: 0.2414
Epoch 52/80
8000/8000 [==============================] - 0s 8us/step - loss: 0.2611 - mean_absolute_error: 0.0722 - val_loss: 0.7525 - val_mean_absolute_error: 0.2398
Epoch 53/80
8000/8000 [==============================] - 0s 6us/step - loss: 0.2616 - mean_absolute_error: 0.0720 - val_loss: 0.7626 - val_mean_absolute_error: 0.2418
Epoch 54/80
8000/8000 [==============================] - 0s 6us/step - loss: 0.2604 - mean_absolute_error: 0.0716 - val_loss: 0.7570 - val_mean_absolute_error: 0.2408
Epoch 55/80
8000/8000 [==============================] - 0s 7us/step - loss: 0.2590 - mean_absolute_error: 0.0701 - val_loss: 0.7613 - val_mean_absolute_error: 0.240

In [58]:
docs = [
    "I really hate this",
    "I don't like this",
    "I think this is nice",
    "I love this thing so much",
    "I feel relatively indifferent",]
for doc in docs:
    result = model.predict(np.asmatrix(nlp(doc).vector))
    print(result)

[[0.15353452 0.02435592]]
[[0.07771567 0.00667944]]
[[0.5506569 0.7326522]]
[[0.00711581 0.6554239 ]]
[[0.00308359 0.8789249 ]]
